In [132]:
%reload_ext autoreload
%autoreload 2

In [133]:
import pymongo
from pymongo import MongoClient
from vastai.api import VastClient
from toolbench.tools import print_module_versions
print_module_versions(globals())

             IPython  {'7.8.0'}
            paramiko  {'2.6.0'}
             pymongo  {'3.9.0'}
           toolbench  {'0.0.1'}
             plumbum  {(1, 6, 7)}
            hyperopt  {'0.2.1'}
 avian_vocalizations  {'0.0.1'}


In [134]:
vast = VastClient().authenticate()

api_key_file:  ~/.vast_api_key
Initializing vast.ai client with api_key from /home/sam/.vast_api_key.
Already logged in.


## Startup and check ssh connection

In [141]:
mongo_instance = vast.get_instance(400808)
mongo_instance.start().wait_until_running(check_every_s=15)
mongo_machine = mongo_instance.pb_remote
mongo_machine

Starting instance 400808.


<ParamikoMachine paramiko://root@ssh4.vast.ai>

In [140]:
mongo_instance.ssh_connection_command

'ssh root@ssh5.vast.ai -p 10787 -i /home/sam/.ssh/vastai'

In [6]:
mongo_instance.pb_remote['pwd']()

Session is alive? True


'/root\n'

In [5]:
print(mongo_machine['pwd']())

/root



In [6]:
mongo_instance.pb_remote['pwd']()

'/root\n'

In [11]:
print(mongo_machine['tail']('/var/log/mongodb/mongodb.log'))

2019-10-24T13:15:48.810+0000 I NETWORK  [conn4] end connection 127.0.0.1:49614 (2 connections now open)
2019-10-24T13:16:30.326+0000 I NETWORK  [listener] connection accepted from 127.0.0.1:49624 #7 (1 connection now open)
2019-10-24T13:16:30.493+0000 I NETWORK  [listener] connection accepted from 127.0.0.1:49626 #8 (2 connections now open)
2019-10-24T13:16:30.728+0000 I NETWORK  [conn7] received client metadata from 127.0.0.1:49624 conn: { driver: { name: "PyMongo", version: "3.9.0" }, os: { type: "Linux", name: "Linux", architecture: "x86_64", version: "3.8.11" }, platform: "CPython 3.6.8.final.0" }
2019-10-24T13:16:31.096+0000 I NETWORK  [conn8] received client metadata from 127.0.0.1:49626 conn: { driver: { name: "PyMongo", version: "3.9.0" }, os: { type: "Linux", name: "Linux", architecture: "x86_64", version: "3.8.11" }, platform: "CPython 3.6.8.final.0" }
2019-10-24T13:16:32.885+0000 I NETWORK  [listener] connection accepted from 127.0.0.1:49628 #9 (3 connections now open)
2019-

## Time SshMachine vs ParamikoMachine

In [15]:
%timeit ssh['ls']()

6.57 s ± 308 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [16]:
%timeit mongo_machine['ls']()

3.85 s ± 174 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Test tunnel to mongodb

In [142]:
mongo_instance = vast.get_instance(400808)

In [143]:
mongo_instance.ssh_connection_command

'ssh root@ssh4.vast.ai -p 10808 -i /home/sam/.ssh/vastai'

In [37]:
with mongo_instance.get_tunnel(27017,27017) as tunn:
    mongo = MongoClient('localhost', 27017)
    print(mongo.list_database_names())

['admin', 'local']


In [18]:
mongo_instance._tunnels

{27017: <SshTunnel (defunct)>}

In [22]:
mongo_instance._ssh_machine_alive

True

## Hot update evaluator code

In [178]:
mongo_instance.pb_remote.sftp.put('avian_vocalizations/evaluator.py',
                                  '/root/avian_vocalizations/avian_vocalizations/evaluator.py')

<SFTPAttributes: [ size=4563 uid=0 gid=0 mode=0o100644 atime=1572063359 mtime=1572064088 ]>

In [65]:
mongo_instance.pb_remote.sftp.put('setup.py','/root/avian_vocalizations/setup.py')

<SFTPAttributes: [ size=1821 uid=0 gid=0 mode=0o100644 atime=1571971573 mtime=1571976662 ]>

In [154]:
mongo_instance.pb_remote.sftp.put('avian_vocalizations/data.py','/root/avian_vocalizations/avian_vocalizations/data.py')

<SFTPAttributes: [ size=13818 uid=0 gid=0 mode=0o100644 atime=1572062804 mtime=1572063000 ]>

In [72]:
mongo_instance.pb_remote.sftp.put('avian_vocalizations/model.py','/root/avian_vocalizations/avian_vocalizations/model.py')

<SFTPAttributes: [ size=3165 uid=0 gid=0 mode=0o100644 atime=1571985722 mtime=1571985722 ]>

## Run some trials

In [144]:
tunnel = mongo_instance.get_tunnel(27017, 27017)

In [145]:
trials = MongoTrials('mongo://localhost:27017/hyperopt/jobs', exp_key='exp%03i'%EXP_NUM)

In [ ]:
# with mongo_instance.pb_remote.popen("pipenv run hyperopt-mongo-worker "
#                                     "--mongo=localhost:27017/hyperopt "
#                                     "--max-consecutive-failures=1 "
#                                     "--reserve-timeout=36600"
#                                    ) as :

In [ ]:
hp_space = ParamSpace(n_frames=128, dropout_rate=hp.uniform('dropout',0,.5),batch_size=64)

evaluator = EvaluatorFactory(n_splits=3, n_epochs=10, data_dir='data')
best = fmin(evaluator, hp_space, trials=trials, algo=tpe.suggest, max_evals=5 )


85it [01:38,  1.13s/it, best loss: ?]                      

In [153]:
os.path.join('~','','.ssh')

'~/.ssh'

In [181]:
trials.delete_all()

## Cleanup

In [ ]:
tunnel.close()